## Step 12: 가변 길이 인수(개선 편)

앞서 구현한 가변 길이 인수에 대응하는 Add 클래스에는 개선 사항이 필요하다. 이번 단계에서 사용자를 위한 개선과 구현자를 위한 개선 각 하나씩을 추가하도록 하겠다.

### 12.1 첫 번째 개선: 함수를 사용하기 쉽게

앞서 구현한 Add 클래스에서는 사용자에게 입력 변수를 리스트로 만들 것을 요구했다. 이번에는 각각을 바로 입력 변수로 넣으면 수행될 수 있게 해보도록 하자.

```python
class Function:
    def __call__(self, *inputs): # 가변 길이 인수
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs

        # 리스트를 반환 / 원소가 한 개 라면 첫 번째 원소 반환
        return outputs if len(outputs) > 1 else outputs[0]
```

여기서 눈에 띄는 부분은 __call__ 메서드 인수 부분의 `*inputs`이다. 함수를 정의할 때 인수 앞에 *를 붙여줌으로써 리스트 대신 임의 개수의 인수를 건네 받을 수 있게 된다.

```python
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))'
f = Add()

y = f(x0, x1)
print(y.data)
``` 

### 12.2 두 번째 개선: 함수를 구현하기 쉽도록

현재의 Add 클래스는 forward 메서드에서 입력 변수를 unpacking 시켜주는 과정이 필요하다. 이를 좀더 개선하면 변수를 입력 받아 변수를 사용할 수 있도록 구현할 수 있다.

```python
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # __call__ 에서 unpack
        if not isinstance(ys, tuple): # 튜플이 아닌 경우
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]
```

함수 호출 단계에서 unpack을 수행함으로써 Add 단계에서는 구현이 간소화된다. 이는 Add 외 다른 Function을 상속 받는 함수들을 만들어서 사용할 때 유용할 것이다.

이로써 Add 클래스의 구현은 다음과 같이 간소화된다.

```python
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
```

순전파 메서드를 `def forward(self, x0 x1):`으로 정의할 수 있게 되었다. 결과 또한 y 변수 하나만을 반환하는 구조를 만들어, 변수를 입력 받아 변수를 반환하도록 하였다.

### 12.3 add 함수 구현

마지막으로 Add 클래스를 파이썬 함수로 사용할 수 있게 하겠다.

```python
def add(x0, x1):
    return Add()(x0, x1)
```

```python
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)
```

이상으로 함수가 가변 길이 인수를 더 자연스럽게 다룰 수 있도록 하였다. 이와 같은 방식으로 곱셈 나눗셈 등 다른 순전파 계산도 수행할 수 있다.

다만 역전파에서도 가변 길이 인수를 다루기 위해서는 다른 구현 방법이 필요하다. 이는 다음 단계에서 다루도록 하겠다.